<a href="https://colab.research.google.com/github/ThorGroth/D-D/blob/master/DnD_1_9_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# version 1.0


In [ ]:
import random

# Elemente mit Stärken und Schwächen
elemente = {
    "Feuer": "Wind",
    "Wasser": "Feuer",
    "Erde": "Wasser",
    "Wind": "Erde"
}

# Klassen für Charaktere und Gegner
class Character:
    def __init__(self, name, char_class, hp, attack, defense, magic, element):
        self.name = name
        self.char_class = char_class
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.magic = magic
        self.element = element
        self.xp = 0
        self.level = 1
        self.inventory = []

    def take_damage(self, damage, enemy_element):
        if enemy_element and elemente.get(enemy_element) == self.element:
            damage *= 1.5  # Extra Schaden bei Elementarschwäche
        actual_damage = max(0, damage - self.defense)
        self.hp -= actual_damage
        self.hp = max(0, self.hp)
        print(f"{self.name} erleidet {actual_damage} Schaden. ({self.hp} HP übrig)")

    def is_alive(self):
        return self.hp > 0

    def attack_enemy(self, enemy):
        damage = random.randint(1, self.attack)
        print(f"{self.name} greift {enemy.name} an und verursacht {damage} Schaden.")
        enemy.take_damage(damage, self.element)

    def heal(self):
        healing = random.randint(10, 20)
        self.hp += healing
        print(f"{self.name} heilt sich um {healing} HP. ({self.hp} HP insgesamt)")

    def level_up(self):
        self.level += 1
        self.attack += 5
        self.hp += 20
        self.defense += 2
        self.magic += 1
        self.xp = 0
        print(f"{self.name} erreicht Level {self.level}! Neue Fähigkeiten erlangt.")

    def equip_item(self, item):
        self.inventory.append(item)
        self.attack += item.attack_bonus
        self.defense += item.defense_bonus
        print(f"{self.name} hat {item.name} ausgerüstet!")

class Enemy:
    def __init__(self, name, hp, attack, defense, element):
        self.name = name
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.element = element

    def take_damage(self, damage, enemy_element):
        if enemy_element and elemente.get(enemy_element) == self.element:
            damage *= 1.5
        actual_damage = max(0, damage - self.defense)
        self.hp -= actual_damage
        self.hp = max(0, self.hp)
        print(f"{self.name} erleidet {actual_damage} Schaden. ({self.hp} HP übrig)")

    def is_alive(self):
        return self.hp > 0

    def attack_player(self, player):
        damage = random.randint(1, self.attack)
        print(f"{self.name} greift {player.name} an und verursacht {damage} Schaden.")
        player.take_damage(damage, self.element)

class Item:
    def __init__(self, name, attack_bonus, defense_bonus):
        self.name = name
        self.attack_bonus = attack_bonus
        self.defense_bonus = defense_bonus

class Player(Character):
    def __init__(self, name, char_class, element):
        if char_class == "Krieger":
            super().__init__(name, char_class, 120, 15, 5, 0, element)
        elif char_class == "Magier":
            super().__init__(name, char_class, 80, 10, 3, 3, element)
        elif char_class == "Dieb":
            super().__init__(name, char_class, 100, 12, 4, 2, element)
        else:
            super().__init__(name, "Krieger", 120, 15, 5, 0, element)
        self.gold = 100

class Shop:
    def __init__(self):
        self.items = {
            "Schwert": Item("Schwert", 10, 0),
            "Rüstung": Item("Rüstung", 0, 5),
            "Magischer Ring": Item("Magischer Ring", 5, 2)
        }

    def show_items(self):
        print("\nWillkommen im Shop!")
        for item in self.items.values():
            print(f"{item.name}: Angriff +{item.attack_bonus}, Verteidigung +{item.defense_bonus}")

    def buy_item(self, item_name, player):
        if item_name in self.items:
            item = self.items[item_name]
            player.equip_item(item)
        else:
            print("Dieses Item gibt es nicht.")

def start_game():
    print("Willkommen zu Dungeons & Dragons!")
    name = input("Gib deinen Charakter-Namen ein: ")
    char_class = input("Wähle deine Klasse (Krieger, Magier, Dieb): ").capitalize()
    element = input("Wähle dein Element (Feuer, Wasser, Erde, Wind): ").capitalize()

    player = Player(name, char_class, element)
    shop = Shop()
    enemies = [Enemy("Goblin", 50, 10, 2, "Erde"), Enemy("Wasserschlange", 60, 12, 3, "Wasser"), Enemy("Feuerdämon", 80, 15, 5, "Feuer"), Enemy("Windgeist", 70, 14, 4, "Wind")]

    print(f"\nDu bist ein {player.char_class} mit {player.hp} HP, Angriff: {player.attack}, Verteidigung: {player.defense}, Magie: {player.magic}, Element: {player.element}.")
    print("Du betrittst den Dungeon...\n")

    while player.is_alive():
        enemy = random.choice(enemies)
        print(f"\nDu triffst auf einen {enemy.name} ({enemy.hp} HP, Element: {enemy.element})!")
        action = input("\nWas möchtest du tun? (Angreifen / Heilen / Shop / Fliehen): ").lower()

        if action == "angreifen":
            player.attack_enemy(enemy)
        elif action == "heilen":
            player.heal()
        elif action == "shop":
            shop.show_items()
            item_choice = input("Was möchtest du kaufen? (Name des Items): ").capitalize()
            shop.buy_item(item_choice, player)
        elif action == "fliehen":
            print("Du versuchst zu fliehen...")
            if random.random() > 0.5:
                print("Flucht erfolgreich!")
                continue
            else:
                print("Die Flucht ist fehlgeschlagen!")
        else:
            print("Ungültige Aktion!")

        if enemy.is_alive():
            enemy.attack_player(player)
        else:
            print(f"\nDu hast den {enemy.name} besiegt!")
            player.gain_xp(50)

    print(f"\n{player.name} wurde besiegt... Spiel vorbei!")

if __name__ == "__main__":
    start_game()


Willkommen zu Dungeons & Dragons!


KeyboardInterrupt: Interrupted by user

# version1.1

In [ ]:
import random

# Elemente mit Stärken und Schwächenj
elemente = {
    "Feuer": "Wind",
    "Wasser": "Feuer",
    "Erde": "Wasser",
    "Wind": "Erde"
}

# Einführungsgeschichte
def introduction():
    print("Willkommen, mutiger Abenteurer! Das Königreich Eldoria liegt in Dunkelheit,\n seit der finstere Magier Malakar die legendären Artefakte gestohlen hat.\n Deine Aufgabe ist es, durch verwunschene Wälder,\n uralte Ruinen und magische Portale zu reisen, um die Artefakte zurückzubringen.\n Auf deinem Weg triffst du auf gefährliche Gegner, aber auch auf magische Wesen,\n die dir helfen können. Bist du bereit für dieses epische Abenteuer?")

# Klassen für Charaktere und Gegner
class Character:
    def __init__(self, name, char_class, hp, attack, defense, magic, element):
        self.name = name
        self.char_class = char_class
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.magic = magic
        self.element = element
        self.xp = 0
        self.level = 1
        self.inventory = []
        self.gold = 50

    def take_damage(self, damage, enemy_element):
        if enemy_element and elemente.get(enemy_element) == self.element:
            damage *= 1.5
        actual_damage = max(0, damage - self.defense)
        self.hp -= actual_damage
        self.hp = max(0, self.hp)
        print(f"{self.name} erleidet {actual_damage} Schaden. ({self.hp} HP übrig)")

    def is_alive(self):
        return self.hp > 0

    def attack_enemy(self, enemy):
        damage = random.randint(1, self.attack)
        print(f"{self.name} greift {enemy.name} an und verursacht {damage} Schaden.")
        enemy.take_damage(damage, self.element)

    def heal(self):
        healing = random.randint(10, 20)
        self.hp += healing
        print(f"{self.name} heilt sich um {healing} HP. ({self.hp} HP insgesamt)")

    def use_magic(self, enemy):
        if self.magic > 0:
            magic_damage = random.randint(10, 25)
            self.magic -= 1
            print(f"{self.name} nutzt Magie und verursacht {magic_damage} Schaden an {enemy.name}.")
            enemy.take_damage(magic_damage, self.element)
        else:
            print("Keine Magie mehr verfügbar!")

    def level_up(self):
        self.level += 1
        self.attack += 5
        self.hp += 20
        self.defense += 2
        self.magic += 1
        self.xp = 0
        print(f"{self.name} erreicht Level {self.level}! Neue Fähigkeiten erlangt.")

    def gain_xp(self, amount):
        self.xp += amount
        print(f"{self.name} erhält {amount} XP.")
        if self.xp >= 100:
            self.level_up()

class Enemy:
    def __init__(self, name, hp, attack, defense, element):
        self.name = name
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.element = element

    def take_damage(self, damage, enemy_element):
        if enemy_element and elemente.get(enemy_element) == self.element:
            damage *= 1.5
        actual_damage = max(0, damage - self.defense)
        self.hp -= actual_damage
        self.hp = max(0, self.hp)
        print(f"{self.name} erleidet {actual_damage} Schaden. ({self.hp} HP übrig)")

    def is_alive(self):
        return self.hp > 0

    def attack_player(self, player):
        damage = random.randint(1, self.attack)
        print(f"{self.name} greift {player.name} an und verursacht {damage} Schaden.")
        player.take_damage(damage, self.element)

class Shop:
    @staticmethod
    def visit_shop(player):
        print("\nWillkommen im Shop! Hier kannst du Items kaufen.")
        choice = input("Möchtest du ein Item kaufen? (J/N): ").lower()
        if choice == "j":
            print("Leider ist der Shop noch nicht implementiert!")
        else:
            print("Du verlässt den Shop.")

class Event:
    @staticmethod
    def random_event(player):
        events = [
            Event.magic_fountain,
            Event.treasure_chest,
            Event.friendly_fairy,
            Event.lost_wanderer
        ]
        random.choice(events)(player)

    @staticmethod
    def magic_fountain(player):
        print("\nDu entdeckst einen magischen Brunnen!")
        if random.random() < 0.3:
            choice = input("Möchtest du daraus trinken? (J/N): ").lower()
            if choice == "j":
                healing = random.randint(20, 40)
                player.hp += healing
                print(f"Das Wasser heilt dich um {healing} HP. ({player.hp} HP insgesamt)")
        else:
            print("Doch es ist nur eine Fata Morgana!")

    @staticmethod
    def treasure_chest(player):
        print("\nDu findest eine alte Schatztruhe!")
        reward = random.choice(["Gold", "Heiltrank", "Magischer Kristall", "Verzaubertes Schwert"])
        print(f"Du öffnest die Truhe und findest: {reward}!")

    @staticmethod
    def friendly_fairy(player):
        print("\nEine strahlende Fee erscheint vor dir!")
        player.magic += 1
        print("Die Fee verstärkt deine magischen Fähigkeiten!")

    @staticmethod
    def lost_wanderer(player):
        print("\nEin verlorener Wanderer bittet um Hilfe.")
        choice = input("Möchtest du ihm helfen? (J/N): ").lower()
        if choice == "j":
            player.gold += 20
            print("Der Wanderer belohnt dich mit Gold!")
        else:
            print("Der Wanderer geht traurig davon.")

if __name__ == "__main__":
    start_game()


# version 1.2

In [ ]:
import random

# Elemente mit Stärken und Schwächen
elemente = {
    "Feuer": "Wind",
    "Wasser": "Feuer",
    "Erde": "Wasser",
    "Wind": "Erde"
}

# Einführungsgeschichte
def introduction():
    print("Willkommen, mutiger Abenteurer! Das Königreich Eldoria liegt in Dunkelheit,\n seit der finstere Magier Malakar die legendären Artefakte gestohlen hat.\n Deine Aufgabe ist es, durch verwunschene Wälder,\n uralte Ruinen und magische Portale zu reisen, um die Artefakte zurückzubringen.\n Auf deinem Weg triffst du auf gefährliche Gegner, aber auch auf magische Wesen,\n die dir helfen können. Bist du bereit für dieses epische Abenteuer?")

# Klassen für Charaktere und Gegner
class Character:
    def __init__(self, name, char_class, hp, attack, defense, magic, element):
        self.name = name
        self.char_class = char_class
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.magic = magic
        self.element = element
        self.xp = 0
        self.level = 1
        self.inventory = []
        self.gold = 50
        self.story_progress = 0  # Fortschritt der Geschichte

    def take_damage(self, damage, enemy_element):
        if enemy_element and elemente.get(enemy_element) == self.element:
            damage *= 1.5
        actual_damage = max(0, damage - self.defense)
        self.hp -= actual_damage
        self.hp = max(0, self.hp)
        print(f"{self.name} erleidet {actual_damage} Schaden. ({self.hp} HP übrig)")

    def is_alive(self):
        return self.hp > 0

    def attack_enemy(self, enemy):
        damage = random.randint(1, self.attack)
        print(f"{self.name} greift {enemy.name} an und verursacht {damage} Schaden.")
        enemy.take_damage(damage, self.element)

    def heal(self):
        healing = random.randint(10, 20)
        self.hp += healing
        print(f"{self.name} heilt sich um {healing} HP. ({self.hp} HP insgesamt)")

    def use_magic(self, enemy):
        if self.magic > 0:
            magic_damage = random.randint(10, 25)
            self.magic -= 1
            print(f"{self.name} nutzt Magie und verursacht {magic_damage} Schaden an {enemy.name}.")
            enemy.take_damage(magic_damage, self.element)
        else:
            print("Keine Magie mehr verfügbar!")

    def level_up(self):
        self.level += 1
        self.attack += 5
        self.hp += 20
        self.defense += 2
        self.magic += 1
        self.xp = 0
        print(f"{self.name} erreicht Level {self.level}! Neue Fähigkeiten erlangt.")

    def gain_xp(self, amount):
        self.xp += amount
        print(f"{self.name} erhält {amount} XP.")
        if self.xp >= 100:
            self.level_up()

    def progress_story(self):
        self.story_progress += 1  # Fortschritt erhöhen
        print(f"Dein Fortschritt: {self.story_progress}/5")

class Enemy:
    def __init__(self, name, hp, attack, defense, element):
        self.name = name
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.element = element

    def take_damage(self, damage, enemy_element):
        if enemy_element and elemente.get(enemy_element) == self.element:
            damage *= 1.5
        actual_damage = max(0, damage - self.defense)
        self.hp -= actual_damage
        self.hp = max(0, self.hp)
        print(f"{self.name} erleidet {actual_damage} Schaden. ({self.hp} HP übrig)")

    def is_alive(self):
        return self.hp > 0

    def attack_player(self, player):
        damage = random.randint(1, self.attack)
        print(f"{self.name} greift {player.name} an und verursacht {damage} Schaden.")
        player.take_damage(damage, self.element)

class Shop:
    @staticmethod
    def visit_shop(player):
        print("\nWillkommen im Shop! Hier kannst du Items kaufen.")
        choice = input("Möchtest du ein Item kaufen? (J/N): ").lower()
        if choice == "j":
            print("Leider ist der Shop noch nicht implementiert!")
        else:
            print("Du verlässt den Shop.")

class Event:
    @staticmethod
    def random_event(player):
        if player.story_progress < 5:  # Zufällige Begegnungen nur, wenn der Spieler noch nicht weit fortgeschritten ist
            events = [
                Event.magic_fountain,
                Event.treasure_chest,
                Event.friendly_fairy,
                Event.lost_wanderer
            ]
            random.choice(events)(player)
        else:
            print("Du hast bereits genug Abenteuer erlebt, deine Reise geht weiter!")

    @staticmethod
    def magic_fountain(player):
        print("\nDu entdeckst einen magischen Brunnen!")
        if random.random() < 0.3:
            choice = input("Möchtest du daraus trinken? (J/N): ").lower()
            if choice == "j":
                healing = random.randint(20, 40)
                player.hp += healing
                print(f"Das Wasser heilt dich um {healing} HP. ({player.hp} HP insgesamt)")

    @staticmethod
    def treasure_chest(player):
        print("\nDu findest eine alte Schatztruhe!")
        reward = random.choice(["Gold", "Heiltrank", "Magischer Kristall", "Verzaubertes Schwert"])
        print(f"Du öffnest die Truhe und findest: {reward}!")

    @staticmethod
    def friendly_fairy(player):
        print("\nEine strahlende Fee erscheint vor dir!")
        player.magic += 1
        print("Die Fee verstärkt deine magischen Fähigkeiten!")

    @staticmethod
    def lost_wanderer(player):
        print("\nEin verlorener Wanderer bittet um Hilfe.")
        choice = input("Möchtest du ihm helfen? (J/N): ").lower()
        if choice == "j":
            player.gold += 20
            print("Der Wanderer belohnt dich mit Gold!")
        else:
            print("Der Wanderer geht traurig davon.")

def start_game():
    introduction()
    player_name = input("Gib deinem Charakter einen Namen: ")
    player = Character(player_name, "Krieger", 100, 20, 10, 5, "Feuer")

    while player.is_alive():
        print("\nWas möchtest du tun?")
        print("1. Einen Gegner angreifen")
        print("2. Magie einsetzen")
        print("3. Heilen")
        print("4. Shop besuchen")
        print("5. Weiterreisen")
        action = input("Wähle eine Aktion (1/2/3/4/5): ")

        if action == "1":
            # Gegner wird erstellt und angegriffen
            enemy = Enemy("Dunkler Krieger", 50, 15, 5, "Wasser")
            player.attack_enemy(enemy)
            if enemy.is_alive():
                enemy.attack_player(player)
            player.gain_xp(10)
        elif action == "2":
            player.use_magic(enemy)
        elif action == "3":
            player.heal()
        elif action == "4":
            Shop.visit_shop(player)
        elif action == "5":
            player.progress_story()
            Event.random_event(player)
        else:
            print("Ungültige Eingabe. Bitte wähle eine gültige Aktion.")

if __name__ == "__main__":
    start_game()
